<a href="https://colab.research.google.com/github/DomTHK/DLO_Lab/blob/main/Abgabe/DLO_Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
# https://drive.google.com/drive/folders/1hNFbb-Ksc9Qdyx0_CB7tTJFy7-UNwYwy?usp=sharing 

# Bildgröße 300x200 RGB

from google.colab import drive
import torchvision 
from torchvision import transforms

from glob import glob

from timeit import default_timer as timer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.optim import SGD
from torch.utils.data import Dataset, random_split
from torchvision import datasets, transforms


In [152]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [153]:
# Config Feld
path = f"/content/drive/MyDrive/Colab Notebooks/data/*/*/*.png"

device = "cuda" if torch.cuda.is_available() else "cpu"
TrainingMode = True

valid_size = 0.1
test_size = 0.2

In [154]:
#  transforms.CenterCrop(224),
transform = transforms.Compose([transforms.Resize((300, 200)),
                                
                                 transforms.ToTensor()])

dataset = datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/data', transform=transform)

In [164]:
class RSPDataset(Dataset):
    def __init__(self, x, y):
        x = x.float()/255
        x = x.view(-1, 300*200)
        self.x, self.y = x, y

    def __getitem__(self, ix):
        x, y = self.x[ix], self.y[ix]
        return x.to(device), y.to(device)

    def __len__(self):
        return len(self.x)

In [165]:
# https://discuss.pytorch.org/t/how-to-use-imagefolder-with-list-of-images-for-train-and-test-sets/120247
# trainset = torch.utils.data.Subset(dataset_train, train_indices)
# valset = torch.utils.data.Subset(dataset_val, val_indices)
def get_data(): 
  n_val = int(np.floor(valid_size * len(dataset)))
  n_test = int(np.floor(test_size * len(dataset)))
  n_train = len(dataset) - n_val - n_test

  train_ds, val_ds, test_ds = random_split(dataset, [n_train, n_val, n_test])

  train = RSPDataset(train_ds.dataset.data, train_ds.dataset.targets)
  
  train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
  valid_dl = torch.utils.data.DataLoader(val_ds, batch_size=32, shuffle=True)
  test_dl = torch.utils.data.DataLoader(test_ds, batch_size=32, shuffle=True)
  return train_dl, test_dl, valid_dl

In [156]:
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [157]:
@torch.no_grad()
def accuracy(x, y, model):
    with torch.no_grad():
        prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

In [158]:
@torch.no_grad()
def loss(x, y, model):
    prediction = model(x)
    loss = loss_fn(prediction, y)
    return loss.item()

In [159]:
def get_model():
    model = nn.Sequential(
        nn.Linear(300 * 200, 200),  # 28 * 28 Input-Nodes | 40 Output-Nodes
        nn.Sigmoid(),  # Aktivierungsfunktion
        nn.Linear(200, 40),
        nn.Sigmoid(),
        nn.Dropout(p=0.2),
        nn.Linear(40, 40),
        nn.Sigmoid(),  # Aktivierungsfunktion
        nn.Linear(40, 20),
        nn.Sigmoid(),
        nn.Linear(20, 3)  # 20 Input-Nodes | 10 Output-Nodes
    ).to(device)
    loss_fn = nn.CrossEntropyLoss()  # Kreuzentropie
    optimizer = SGD(model.parameters(), lr=1e-1)
    return model, loss_fn, optimizer

In [160]:
def train_batch(x, y, model, opt, loss_fn):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

In [161]:
model, loss_fn, optimizer = get_model()

In [166]:
train_dl, test_dl, val_dl = get_data()

AttributeError: ignored

In [163]:
print('Starting training...')

time0 = timer()

epochs = 50

arrPlotX = []
train_losses, train_accuracies = [], []
test_losses, test_accuracies = [], []

if TrainingMode:
    for epoch in range(epochs):
        train_epoch_losses, train_epoch_accuracies = [], []

        for ix, batch in enumerate(iter(train_dl)):
            x, y = batch
            batch_loss = train_batch(x, y, model, optimizer, loss_fn)  # Mini-Batch
            train_epoch_losses.append(batch_loss)
        train_epoch_loss = np.array(train_epoch_losses).mean()

        for ix, batch in enumerate(iter(train_dl)):
            x, y = batch
            is_correct = accuracy(x, y, model)
            train_epoch_accuracies.extend(is_correct)
        train_epoch_accuracy = np.mean(train_epoch_accuracies)

        val_is_correct = None
        validation_loss = None

        for ix, batch in enumerate(iter(val_dl)):
            x, y = batch
            val_is_correct = accuracy(x, y, model)
            validation_loss = loss(x, y, model)

        val_epoch_accuracy = np.mean(val_is_correct)
        arrPlotX.append(epoch)
        train_losses.append(train_epoch_loss)
        train_accuracies.append(train_epoch_accuracy)
        test_losses.append(validation_loss)
        test_accuracies.append(val_epoch_accuracy)

        # Early stopping
        if val_epoch_accuracy > 0.95:
            break

        print(f"epoch: {epoch}  train_acc: {100 * train_epoch_accuracy:.2f}%  val_acc: {100 * val_epoch_accuracy:.2f}%")

    torch.save(model.state_dict(), ModelPath)

    plt.plot(arrPlotX, train_accuracies)
    plt.plot(arrPlotX, test_accuracies)
    plt.savefig("accuracies.png")
else:
    val_is_correct = None
    for ix, batch in enumerate(iter(test_dl)):
        x, y = batch
        val_is_correct = accuracy(x, y, model)
        validation_loss = loss(x, y, model)

    for ix, var in enumerate(iter(val_is_correct)):
        if var is False:
            print(test_dl.dataset.__getitem__(ix))

    val_epoch_accuracy = np.mean(val_is_correct)
    print(f"test_acc: {100 * val_epoch_accuracy:.2f}%")


Starting training...


RuntimeError: ignored